<a href="https://colab.research.google.com/github/Asia99-91/LMS/blob/python-project/PythonProject1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
# Book Class
class Book:
    def __init__(self, title, author, isbn, copies=1):
        self.title = title
        self.author = author
        self.isbn = isbn
        self.copies = copies

    def display_details(self):
        return f"Title: {self.title}, Author: {self.author}, ISBN: {self.isbn}, Copies: {self.copies}"

    def borrow_book(self):
        if self.copies > 0:
            self.copies -= 1
            return True
        return False

    def return_book(self):
        self.copies += 1




In [6]:

# User Class
class User:
    def __init__(self, user_id, name, role='Regular'):
        self.user_id = user_id
        self.name = name
        self.role = role
        self.borrowed_books = {}
        self.fines = 0

    def borrow_book(self, book):
        self.borrowed_books[book.title] = book
        print(f"{self.name} borrowed {book.title}")

    def return_book(self, book, days_late=0):
        if book.title in self.borrowed_books:
            del self.borrowed_books[book.title]
            fine = days_late * 1  # 1 unit fine per day overdue
            self.fines += fine
            print(f"{self.name} returned {book.title}. Fine: {fine}")

    def display_user_details(self):
        borrowed = ', '.join(self.borrowed_books.keys()) if self.borrowed_books else "None"
        return f"User ID: {self.user_id}, Name: {self.name}, Role: {self.role}, Borrowed Books: {borrowed}, Fines: {self.fines}"

    def pay_fine(self, amount):
        if amount <= self.fines:
            self.fines -= amount
            print(f"{self.name} paid a fine of {amount}. Remaining fines: {self.fines}")
        else:
            print(f"{self.name} tried to overpay. Fines left: {self.fines}")


# Admin User inherits from User
class Admin(User):
    def __init__(self, user_id, name):
        super().__init__(user_id, name, role='Admin')



In [7]:

# Library Class
class Library:
    def __init__(self):
        self.books = []
        self.users = {}
        self.transactions = []
        self.reservations = {}

    def add_book(self, book):
        self.books.append(book)
        print(f"Book '{book.title}' added to library.")

    def remove_book(self, book_title):
        self.books = [book for book in self.books if book.title != book_title]
        print(f"Book '{book_title}' removed from library.")

    def display_books(self):
        if not self.books:
            print("No books available in the library.")
        for book in self.books:
            print(book.display_details())

    def add_user(self, user):
        self.users[user.user_id] = user
        print(f"User '{user.name}' added to system.")

    def display_users(self):
        if not self.users:
            print("No users in the system.")
        for user in self.users.values():
            print(user.display_user_details())

    def borrow_book(self, user_id, book_title):
        user = self.users.get(user_id)
        if not user:
            print(f"No user with ID {user_id} found.")
            return

        for book in self.books:
            if book.title == book_title:
                if book.borrow_book():
                    user.borrow_book(book)
                    self.transactions.append(f"{user.name} borrowed {book.title}")
                    return
                else:
                    if book_title not in self.reservations:
                        self.reservations[book_title] = []
                    self.reservations[book_title].append(user)
                    print(f"Book '{book_title}' is unavailable. {user.name} reserved the book.")
                    return
        print(f"Book '{book_title}' does not exist in the library.")

    def return_book(self, user_id, book_title, days_late=0):
        user = self.users.get(user_id)
        if not user:
            print(f"No user with ID {user_id} found.")
            return

        for book in self.books:
            if book.title == book_title:
                book.return_book()
                user.return_book(book, days_late)
                self.transactions.append(f"{user.name} returned {book.title} with {days_late} days late")

                if book_title in self.reservations and self.reservations[book_title]:
                    next_user = self.reservations[book_title].pop(0)
                    print(f"Book '{book_title}' is now available for {next_user.name}")
                return
        print(f"Book '{book_title}' was not borrowed by {user.name}")

    def log_transactions(self):
        if not self.transactions:
            print("No transactions recorded.")
        else:
            print("Transaction Logs:")
            for transaction in self.transactions:
                print(transaction)

    def pay_fine(self, user_id, amount):
        user = self.users.get(user_id)
        if user:
            user.pay_fine(amount)
        else:
            print(f"No user with ID {user_id} found.")


In [8]:

# Function to display the menu and handle user input
def display_menu():
    print("\nLibrary Management System")
    print("1. Add Book")
    print("2. Add User")
    print("3. Display Books")
    print("4. Display Users")
    print("5. Borrow Book")
    print("6. Return Book")
    print("7. Pay Fine")
    print("8. Display Transaction Logs")
    print("9. Exit")


def main():
    library = Library()

    while True:
        display_menu()
        choice = input("Enter your choice (1-9): ")

        if choice == "1":
            title = input("Enter book title: ")
            author = input("Enter book author: ")
            isbn = input("Enter book ISBN: ")
            copies = int(input("Enter number of copies: "))
            book = Book(title, author, isbn, copies)
            library.add_book(book)

        elif choice == "2":
            user_id = int(input("Enter user ID: "))
            name = input("Enter user name: ")
            role = input("Enter role (Admin/Regular): ")
            if role.lower() == "admin":
                user = Admin(user_id, name)
            else:
                user = User(user_id, name)
            library.add_user(user)

        elif choice == "3":
            library.display_books()

        elif choice == "4":
            library.display_users()

        elif choice == "5":
            user_id = int(input("Enter your user ID: "))
            book_title = input("Enter the book title to borrow: ")
            library.borrow_book(user_id, book_title)

        elif choice == "6":
            user_id = int(input("Enter your user ID: "))
            book_title = input("Enter the book title to return: ")
            days_late = int(input("Enter the number of days late (if any): "))
            library.return_book(user_id, book_title, days_late)

        elif choice == "7":
            user_id = int(input("Enter your user ID: "))
            amount = int(input("Enter the amount to pay towards fines: "))
            library.pay_fine(user_id, amount)

        elif choice == "8":
            library.log_transactions()

        elif choice == "9":
            print("Exiting the system...")
            break

        else:
            print("Invalid choice. Please try again.")



In [9]:
main()


Library Management System
1. Add Book
2. Add User
3. Display Books
4. Display Users
5. Borrow Book
6. Return Book
7. Pay Fine
8. Display Transaction Logs
9. Exit
Enter your choice (1-9): 4
No users in the system.

Library Management System
1. Add Book
2. Add User
3. Display Books
4. Display Users
5. Borrow Book
6. Return Book
7. Pay Fine
8. Display Transaction Logs
9. Exit
Enter your choice (1-9): 1
Enter book title: جدد عقلك 
Enter book author: عبد الكريم بكار
Enter book ISBN: 1332
Enter number of copies: 3
Book 'جدد عقلك ' added to library.

Library Management System
1. Add Book
2. Add User
3. Display Books
4. Display Users
5. Borrow Book
6. Return Book
7. Pay Fine
8. Display Transaction Logs
9. Exit
Enter your choice (1-9): 2
Enter user ID: 1001
Enter user name: Ahmed
Enter role (Admin/Regular): Admin 
User 'Ahmed' added to system.

Library Management System
1. Add Book
2. Add User
3. Display Books
4. Display Users
5. Borrow Book
6. Return Book
7. Pay Fine
8. Display Transaction Lo